# Benchmark Modeling 01

Now that we've fixed the bug with calculating gene intersection

In [ ]:
import pandas as pd 
import numpy as np

import sys, os 
sys.path.append('../src')

from model import *
from lightning_train import *

In [ ]:
human_data = an.read_h5ad('../data/benchmark/human.h5ad')
mouse_data = an.read_h5ad('../data/benchmark/mouse_clipped.h5ad')

In [ ]:
labels = pd.read_csv('../data/benchmark/human_labels/')
labels.shape

In [ ]:
human_cols = [x.strip().upper() for x in human_data.var.index.values]
mouse_cols = [x.strip().upper() for x in mouse_data.var.index.values]

In [1]:
refgenes = list(set(human_cols).intersection(mouse_cols))
len(refgenes), len(human_cols), len(mouse_cols)

NameError: name 'human_cols' is not defined

In [7]:
'LDB2' in refgenes

True

In [8]:
'CNTN4' in refgenes

True

In [9]:
'DLGAP1' in refgenes

True

In [10]:
'PRKG1' in refgenes

True

In [11]:
'NEGR1' in refgenes

True

In [12]:
module = DataModule(
    datafiles=['../data/benchmark/human.h5ad'],
    labelfiles=['../data/benchmark/human_labels/human_MTG_labels_clean.csv'],
    class_label='subclass_label',
    index_col='cell',
    batch_size=16,
    num_workers=8,
    assume_numeric_label=False,
    currgenes=human_cols,
    refgenes=refgenes,
    preprocess=True,
)

# module.prepare_data()
module.setup()

Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Done, continuing to training.
Calculating weights


In [ ]:
sample = next(iter(module.trainloader))[0]

sample

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/julian/Documents/Projects/SIMS/notebooks/../src/data.py", line 8, in <module>
    import anndata as an 
  File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/anndata/__init__.py", line 7, in <module>
    from ._core.anndata import AnnData
  File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/anndata/_core/anndata.py", line 17, in <module>
    import h5py
  File "/Users/julian/miniconda3/envs/single-cell/lib/python3.9/site-packages/h5py/__init__.py", line 33, in <module>
    from . import version
  File "/Users/julian/miniconda3/envs/single-cell

In [ ]:
sample.shape

In [ ]:
model = SIMSClassifier(
    input_dim=module.input_dim,
    output_dim=module.num_labels,
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=25,
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

wandb_logger = WandbLogger(
    project=f"Benchmark Model, Human MTG",
    name='Human MTG'
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[
        early_stopping_callback,
        lr_callback
    ],
    max_epochs=200,
)

trainer.fit(model, datamodule=module)

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


assume_numeric_label=False, using sklearn.preprocessing.LabelEncoder and encoding target variables.
Transforming labelfile 1/1
Creating train/val/test DataLoaders...


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


...

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:219: UserWarning: You called `self.log('train_confusion_matrix', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

libc++abi: terminating with uncaught exception of type std::length_error: vector


In [8]:
next(iter(module.trainloader))[0][0].shape

torch.Size([16519])